In [5]:
import os
import pandas as pd
import numpy as np

# Load daily data
df = pd.read_csv(r'D:\kowsi\project_works\Advanced_Forecasting_Anomaly_Detection\logic_leap_horizon_datasets\operations_daily_365d.csv')
df['date'] = pd.to_datetime(df['date'])


In [6]:

# Sort by site and date
df = df.sort_values(['site_id', 'date'])

# Calculate rolling mean and std deviation for downtime_minutes for each site with window size 7
df['downtime_roll_mean'] = df.groupby('site_id')['downtime_minutes'].transform(lambda x: x.rolling(7, 1).mean())
df['downtime_roll_std'] = df.groupby('site_id')['downtime_minutes'].transform(lambda x: x.rolling(7, 1).std())

# Calculate Z-score for downtime
df['downtime_zscore'] = (df['downtime_minutes'] - df['downtime_roll_mean']) / df['downtime_roll_std']

# Flag anomalies where absolute Z-score exceeds threshold (e.g., 3)
df['anomaly'] = df['downtime_zscore'].abs() > 3

# Create alerts DataFrame with relevant columns
alerts = df[df['anomaly']][['site_id', 'date', 'downtime_minutes', 'downtime_zscore']]

# Ensure 'outputs' directory exists
output_dir = r'D:\kowsi\project_works\Advanced_Forecasting_Anomaly_Detection\outputs'
os.makedirs(output_dir, exist_ok=True)

# Save alerts to CSV
alerts.to_csv(os.path.join(output_dir, 'alerts.csv'), index=False)

# Display first few rows of alerts
print(alerts.head())


Empty DataFrame
Columns: [site_id, date, downtime_minutes, downtime_zscore]
Index: []
